# Vessel Segmentation
This notebook is an overview of the basic functions required to train and run the vessel segmentation model. It will cover the following:
1. Data preparation
2. Training
3. Prediction
4. Sweeps
5. Writing NIFTI files
6. Validation
7. Others

## 1. Data preparation

The script below will take the training and validation images and their masks and split them into patches of a given size, defined by the width.

- The training and validation **image paths** are put into lists.
    - The masks named "XXXX_mask.png" should be in the same folder as each image.
- Normalization applies a clahe to improve contrast
- The patches are saved in a "train" and "val" folder in the output directory
  <br/> <br/>
- The training images are rotated and flipped for data augmentation
- The validation patches outside the brain area aren't saved

In [ ]:
from liom_toolkit.segmentation.vseg.make_dataset import make_train_val

training = ['data/LSFM_dataset/s23/1350.png', 'data/LSFM_dataset/s23/1000.png', 'data/LSFM_dataset/s23/1500.png',
            'data/LSFM_dataset/s23/700.png', 'data/LSFM_dataset/s23/800.png', 'data/LSFM_dataset/s23/1200.png',
            'data/LSFM_dataset/s23/1110.png', 'data/LSFM_dataset/s23/575.png']
validation = ['data/LSFM_dataset/s23/750.png']
normalization = True
stride = 256
width = 256
output_dir = 'data/patches'

make_train_val(training, validation, normalization, stride, width, output_dir, threshold=0)

## 2. Training
The script below will train the model on the patches created above. 
<br/>
The train_model function can be configurated by the learning rate, batch size and epochs hyperparameters

In [ ]:
from liom_toolkit.segmentation.vseg.training import train_model

In [ ]:
# Set the parameters required for training
dataset_dir = "data/patches"
output = "data/training"
device = "cuda"
learning_rate = 0.003673
batch_size = 35
epochs = 68

In [ ]:
# Train the model
train_model(dataset_dir=dataset_dir, dev=device, output_train=output, learning_rate=learning_rate,
            batch_size=batch_size, epochs=epochs)

## 3. Prediction
The script below will run the model on the images and save the results.<br/>
The model used is the model that was last registered in the Vessel Segmentation registry (this model is tagged @latest)

In [ ]:
from liom_toolkit.segmentation.vseg.predict_one import predict_one
from liom_toolkit.segmentation.vseg.model import VsegModel

device = "cuda"
# Model load
model = VsegModel(pretrained=True, device=device)

### Prediction for one image

- The results are saved in the output_path folder
- The same normalization as for the training can be applied for the prediction
- The results are not great when the images are patched. In the predict_one function, patching can be skipped. Otherwise, if patching is on, a stride and width value have to be given.
- The images are currently saved as png's, with the vessel pixels given a 255 value

In [ ]:
image_path = "data/LSFM/S24/555nm_slices/500.png"
output_path = "data/prediction/s24"
normalization = True
patching = False
stride = None
width = None

In [ ]:
# Run the model
prediction = predict_one(model=model, img_path=image_path, save_path=output_path, norm=normalization, dev=device,
                         patching=patching, stride=stride, width=width)

### Prediction for a folder

- The prediction for every image in the directory is saved in the output folder
- The same paramaters are used as the previous section

In [ ]:
import os
from tqdm.auto import tqdm

dir_path = "data/LSFM/S24/555nm_slices"
output_path = "data/LSFM_predictions"
normalization = True
patching = False
stride = None
width = None

In [ ]:
import time

times = []

# Run the model
for images in tqdm(os.listdir(dir_path)):
    start = time.time()
    if images != ".ipynb_checkpoints":
        image_path = os.path.join(dir_path, images)
        _ = predict_one(model=model, img_path=image_path, save_path=output_path, norm=normalization, dev=device,
                        patching=patching, stride=stride, width=width)
        end = time.time()
        times.append(end - start)

print(f"Total time:{sum(times)}")
print(f"Average time:{sum(times) / len(times)}")

## 4. Sweeps
Sweeps were done to optimize the training batch size, learning rate and number of epochs.

In [ ]:
# Sweep configuration
sweep_config = {'method': 'bayes',
                'metric': {
                    'name': 'Validation Loss',
                    'goal': 'minimize'},
                'parameters': {
                    'batch_size': {
                        'distribution': 'int_uniform',
                        'max': 50,
                        'min': 10
                    },
                    'epochs': {
                        'distribution': 'int_uniform',
                        'max': 75,
                        'min': 10
                    },
                    'learning_rate': {
                        'distribution': 'uniform',
                        'max': 0.005,
                        'min': 5e-6
                    },
                },
                'program': 'training.py'
                }

In [ ]:
sweep_config2 = {'method': 'bayes',
                 'metric': {
                     'name': 'Validation Loss',
                     'goal': 'minimize'},
                 'parameters': {
                     'epochs': {
                         'distribution': 'int_uniform',
                         'max': 75,
                         'min': 10
                     },
                 },
                 'program': 'training.py'
                 }

In [ ]:
# Using the configuration, a sweep ID is created
import wandb

sweep_id = wandb.sweep(sweep_config2, entity="liom-lab", project="vseg")

In [ ]:
# Starts the sweep agent
import wandb
from liom_toolkit.segmentation.vseg import training

sweepid = "liom-lab/vseg/nmwqptw8"  #ID printed from the previous cell
count = 15  # Number of runs 

wandb.agent(sweepid, function=training.train_model, count=count)

## 5. Writing NIFTI files
The following script assembles all the predictions of one volume (inside a folder) into a nifti file. Since predict_one is used, all the individual segmentations are saved in the designated path. <br/>
To do this, every segmentation is added to a list, then stacked as a 3D volume numpy array. The ants library saves it as a NIFTI file

In [ ]:
from liom_toolkit.segmentation.vseg.predict_one import predict_one
from liom_toolkit.segmentation.vseg.model import VsegModel

device = "cuda"
# Model load
model = VsegModel(pretrained=True, device=device)

In [ ]:
import ants
import numpy as np
from tqdm.auto import tqdm
import os
import natsort

In [ ]:
# Prediction for every image in the directory
folder_dir = "data/LSFM/S23/555nm_slices"
output_path = "data/LSFM_predictions"
normalization = True

image_list = os.listdir(folder_dir)
image_list = natsort.natsorted(image_list)
if image_list[-1] == '.ipynb_checkpoints':
    image_list.remove('.ipynb_checkpoints')

image_3D = []

for images in tqdm(image_list):
    image_path = f"{folder_dir}/{images}"
    prediction = predict_one(model=model, img_path=image_path, save_path=output_path, norm=normalization, dev=device,
                             patching=False)
    image_3D.append(prediction)

In [ ]:
volume = np.stack(image_3D)
volume = np.transpose(volume, (2, 1, 0))

In [ ]:
nifti = ants.from_numpy(volume)
nifti.to_file('brainslices.nii')

## 6. Validation
To test the model on new data.

- The  **image paths** are put into a list.
    - The masks named "XXXX_mask.png" should be in the same folder as each image.
    - The images should be in a folder with the volume name
- The normalization is on and the patching is off

The following metrics are calculated:
- Accuracy
- Recall
- Jaccard index
- f1 score

In the save_path folder, the following files are saved:
- The predictions -> "XXXX_segmented.png"
- A comparison of the prediction and the mask -> "volume_XXXX_comparison.png"
    - In this image, the pixels are shown as follows:
      - TP: white
      - TN: black
      - FN: blue
      - FP: red
- A CSV file with the metrics for each image and an average -> "validationmetrics.csv"

In [ ]:
from liom_toolkit.segmentation.vseg.validation import validate_model

images = ["data/LSFM_dataset/s23/750.png", "data/LSFM_dataset/s24/1200.png"]
save_path = "validation"
device = "cuda"

validate_model(model=model, img_list=images, save_path=save_path, device=device)

## 7. Others
Other code left over from the VSEG project

In [ ]:
from skimage.morphology import binary_erosion, disk
from skimage.io import imread, imsave
import numpy as np
from skimage.exposure import equalize_adapthist
from skimage.color import gray2rgb

In [ ]:
# Erodes a mask by one pixel
path = "data/LSFM_dataset/800_mask.png"
output_path = "data/LSFM_eroded_dataset/800_mask.png"
mask = imread(path)
mask = (mask / mask.max()).astype(np.uint8)

erosion_disk = disk(1)

image = binary_erosion(mask, footprint=erosion_disk)
image = image.astype(np.uint8) * 255
imsave(output_path, image, cmap="gray")

In [ ]:
# Converts the png values from 0-255 to 0-1
path = "data/LSFM_dataset/800_mask.png"
image = imread(path)
image = image / image.max()
image = image.astype(np.uint8)
imsave(path, image, check_contrast=False)

In [ ]:
# Saves the Clahe of the selected image

number = 500
volume = "S24"
image_path = f"data/LSFM/{volume}/555nm_slices/{number}.png"
output_path = f"data/clahe/{volume}_{number}_eqhist.png"

image = imread(image_path)
image = (image / image.max() * 255).astype(np.uint8)
image_clahe = equalize_adapthist(image, kernel_size=10, clip_limit=0.05, nbins=128)
image_clahe = gray2rgb(image_clahe)
image_clahe = (image_clahe / image_clahe.max() * 255).astype(np.uint8)

imsave(output_path, image_clahe, cmap="gray")